#Server Monitoring Data Pipeline

## 1.Data Ingestion

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
metadata = pd.read_excel('/content/Data Engineering Use Case Dataset.xlsx',sheet_name='Server_Metadata')

In [ ]:
station1 = pd.read_excel('/content/Data Engineering Use Case Dataset.xlsx',sheet_name='Server_Performance_Station1')

In [ ]:
station2 = pd.read_excel('/content/Data Engineering Use Case Dataset.xlsx',sheet_name='Server_Performance_Station2')

In [ ]:
print(f"Metadata file shape: {metadata.shape}")

Metadata file shape: (100, 9)


In [ ]:
print(f"Station1 file shape: {station1.shape}")

Station1 file shape: (3000, 14)


In [ ]:
print(f"Station2 file shape: {station2.shape}")

Station2 file shape: (2000, 11)


In [ ]:
print(metadata.head())

  Server_ID  Hostname    IP_Address  OS_Type Server_Location Admin_Name  \
0    SRV001  host-001    10.0.186.8  Windows          London     George   
1    SRV002  host-002  10.0.189.206    Linux       Singapore      Alice   
2    SRV003  host-003   10.0.219.47  Windows        New York      Ethan   
3    SRV004  host-004  10.0.185.117    Linux          Mumbai      Fiona   
4    SRV005  host-005   10.0.39.146  Windows        New York      Diana   

  Server_Cluster         Admin_Email      Admin_Phone  
0           CL-8  george@xyzcorp.com  +1-202-749-4405  
1           CL-2   alice@xyzcorp.com  +1-202-531-6896  
2           CL-2   ethan@xyzcorp.com  +1-202-853-1213  
3           CL-6   fiona@xyzcorp.com  +1-202-230-1591  
4           CL-3   diana@xyzcorp.com  +1-202-789-3073  


In [ ]:
print(station1.head())

              Log_ID Server_ID Server_Cluster       Log_Timestamp  \
0  Station1_LOG00001    SRV001           CL-8 2025-08-02 23:30:00   
1  Station1_LOG00002    SRV005           CL-3 2025-08-01 03:12:00   
2  Station1_LOG00003    SRV001           CL-8 2025-08-01 01:33:00   
3  Station1_LOG00004    SRV091           CL-2 2025-08-02 12:56:00   
4  Station1_LOG00005    SRV089           CL-1 2025-08-02 02:50:00   

   CPU_Utilization (%)  Memory_Usage (%)  Disk_IO (%)  \
0                94.63             81.30        44.10   
1                34.98             45.56        58.36   
2                37.41             73.65        74.51   
3                86.86             60.65        23.75   
4                91.59             62.23        50.36   

   Network_Traffic_In (MB/s)  Network_Traffic_Out (MB/s)  Uptime (Hours)  \
0                       1.67                       21.79          506.74   
1                      29.42                         NaN          157.97   
2             

In [ ]:
print(station2.head())

              Log_ID Server_ID Server_Cluster       Log_Timestamp  \
0  Station2_LOG00001    SRV011           CL-7 2025-08-01 10:53:00   
1  Station2_LOG00002    SRV011           CL-7 2025-08-02 09:19:00   
2  Station2_LOG00003    SRV081           CL-4 2025-08-02 07:57:00   
3  Station2_LOG00004    SRV023           CL-1 2025-08-01 16:51:00   
4  Station2_LOG00005    SRV034           CL-6 2025-08-02 22:51:00   

   CPU_Utilization (%)  Memory_Usage (%)  Disk_IO (%)  \
0                40.91             42.64        54.58   
1                10.70               NaN          NaN   
2                59.41             48.17          NaN   
3                45.58             59.19        44.57   
4                55.32             20.39        16.51   

   Network_Traffic_In (MB/s)  Network_Traffic_Out (MB/s)  Uptime (Hours)  \
0                       5.72                       10.61          454.16   
1                       7.42                        7.13          277.75   
2             

## 2. Data Cleaning and Preprocessing

### 2.1 Cleaning the metadata



####Checking nulls

In [ ]:
nulls = metadata.isnull().sum()

In [ ]:
print(nulls)

Server_ID          0
Hostname           0
IP_Address         0
OS_Type            0
Server_Location    0
Admin_Name         0
Server_Cluster     0
Admin_Email        0
Admin_Phone        0
dtype: int64


In [ ]:
id_null = metadata['Server_ID'].isnull().sum()

In [ ]:
print(id_null)

0


#### Ensures no extra spaces

In [ ]:
String_cols= metadata.select_dtypes(include="object").columns


In [ ]:
metadata[String_cols] = metadata[String_cols].apply(lambda x: x.str.strip())

####Checking any duplicates exists

In [ ]:
station1.duplicated().any()

np.False_

###2.2 Cleaning the Station1

####


Checking nulls

In [ ]:
nulls_count = station1.isnull().sum()

In [ ]:
print(nulls_count)

Log_ID                          0
Server_ID                       0
Server_Cluster                  0
Log_Timestamp                   0
CPU_Utilization (%)             0
Memory_Usage (%)              345
Disk_IO (%)                   180
Network_Traffic_In (MB/s)       0
Network_Traffic_Out (MB/s)    130
Uptime (Hours)                  0
Downtime (Hours)                0
Config_Version                  0
Last_Patch_Date                 0
Deployment_Token                0
dtype: int64


####Handling null values of using linear interpolation

In [ ]:
station1['Memory_Usage (%)']= station1['Memory_Usage (%)'].interpolate(method = 'linear')

In [ ]:
station1['Disk_IO (%)']= station1['Disk_IO (%)'].interpolate(method = 'linear')

In [ ]:
station1['Network_Traffic_Out (MB/s)']= station1['Network_Traffic_Out (MB/s)'].interpolate(method = 'linear')

In [ ]:
nulls_count = station1.isnull().sum()
print(nulls_count)

Log_ID                        0
Server_ID                     0
Server_Cluster                0
Log_Timestamp                 0
CPU_Utilization (%)           0
Memory_Usage (%)              0
Disk_IO (%)                   0
Network_Traffic_In (MB/s)     0
Network_Traffic_Out (MB/s)    0
Uptime (Hours)                0
Downtime (Hours)              0
Config_Version                0
Last_Patch_Date               0
Deployment_Token              0
dtype: int64


####checking the Log_Timestamp type

In [ ]:
print(station1["Log_Timestamp"].dtype)


datetime64[ns]


####Checking duplicates

In [ ]:
station1.duplicated().any()

np.False_

In [ ]:
station1[station1.duplicated(subset=['Log_ID'])]

,Log_ID,Server_ID,Server_Cluster,Log_Timestamp,CPU_Utilization (%),Memory_Usage (%),Disk_IO (%),Network_Traffic_In (MB/s),Network_Traffic_Out (MB/s),Uptime (Hours),Downtime (Hours),Config_Version,Last_Patch_Date,Deployment_Token


####Removing unwanted columns

In [ ]:
drop_columns = ['Config_Version','Last_Patch_Date','Deployment_Token']
station1 = station1.drop(columns=drop_columns,errors ='ignore')

In [ ]:
rename={
    'CPU_Utilization (%)':'CPU_Utilization',
    'Memory_Usage (%)':'Memory_Usage',
    'Disk_IO (%)':'Disk_IO',
    'Network_Traffic_In (MB/s)':'Network_Traffic_In_MB/s',
    'Network_Traffic_Out (MB/s)':'Network_Traffic_Out_MB/s',
    'Uptime (Hours)':'Uptime_Hours',
    'Downtime (Hours)':'Downtime_Hours'
}
station1 = station1.rename(columns=rename)

In [ ]:
print(station1.head())

              Log_ID Server_ID Server_Cluster       Log_Timestamp  \
0  Station1_LOG00001    SRV001           CL-8 2025-08-02 23:30:00   
1  Station1_LOG00002    SRV005           CL-3 2025-08-01 03:12:00   
2  Station1_LOG00003    SRV001           CL-8 2025-08-01 01:33:00   
3  Station1_LOG00004    SRV091           CL-2 2025-08-02 12:56:00   
4  Station1_LOG00005    SRV089           CL-1 2025-08-02 02:50:00   

   CPU_Utilization  Memory_Usage  Disk_IO  Network_Traffic_In_MB/s  \
0            94.63         81.30    44.10                     1.67   
1            34.98         45.56    58.36                    29.42   
2            37.41         73.65    74.51                     2.79   
3            86.86         60.65    23.75                    22.67   
4            91.59         62.23    50.36                    11.64   

   Network_Traffic_Out_MB/s  Uptime_Hours  Downtime_Hours  
0                    21.790        506.74            2.77  
1                    20.335        157.97   

####Ensures no extra spaces

In [ ]:
String_cols= metadata.select_dtypes(include="object").columns


In [ ]:
metadata[String_cols] = metadata[String_cols].apply(lambda x: x.str.strip())

###2.3 Cleaning the station2

####Checking Nulls

In [ ]:
nulls_cnt = station2.isnull().sum()

In [ ]:
print(nulls_cnt)

Log_ID                          0
Server_ID                       0
Server_Cluster                  0
Log_Timestamp                   0
CPU_Utilization (%)             0
Memory_Usage (%)              175
Disk_IO (%)                    91
Network_Traffic_In (MB/s)       0
Network_Traffic_Out (MB/s)     78
Uptime (Hours)                  0
Downtime (Hours)                0
dtype: int64


####Handling Null values using linear interpolation

In [ ]:
station2['Memory_Usage (%)']= station2['Memory_Usage (%)'].interpolate(method = 'linear')

In [ ]:
station2['Disk_IO (%)']= station2['Disk_IO (%)'].interpolate(method = 'linear')

####Handling null values using median

In [ ]:
station2['Network_Traffic_Out (MB/s)']=station2.groupby("Server_ID")["Network_Traffic_Out (MB/s)"].transform(lambda x: x.fillna(x.median()))

In [ ]:
nulls_cnt = station1.isnull().sum()
print(nulls_cnt)

Log_ID                      0
Server_ID                   0
Server_Cluster              0
Log_Timestamp               0
CPU_Utilization             0
Memory_Usage                0
Disk_IO                     0
Network_Traffic_In_MB/s     0
Network_Traffic_Out_MB/s    0
Uptime_Hours                0
Downtime_Hours              0
dtype: int64


####Checks for duplicates

In [ ]:
station2.duplicated().any()


np.False_

In [ ]:
station2[station2.duplicated(subset=['Log_ID'])]

,Log_ID,Server_ID,Server_Cluster,Log_Timestamp,CPU_Utilization (%),Memory_Usage (%),Disk_IO (%),Network_Traffic_In (MB/s),Network_Traffic_Out (MB/s),Uptime (Hours),Downtime (Hours)


####Renaming column headers to standard form

In [ ]:
rename={
    'CPU_Utilization (%)':'CPU_Utilization',
    'Memory_Usage (%)':'Memory_Usage',
    'Disk_IO (%)':'Disk_IO',
    'Network_Traffic_In (MB/s)':'Network_Traffic_In_MB/s',
    'Network_Traffic_Out (MB/s)':'Network_Traffic_Out_MB/s',
    'Uptime (Hours)':'Uptime_Hours',
    'Downtime (Hours)':'Downtime_Hours'
}
station2 = station2.rename(columns=rename)

In [ ]:
print(station2.head())

              Log_ID Server_ID Server_Cluster       Log_Timestamp  \
0  Station2_LOG00001    SRV011           CL-7 2025-08-01 10:53:00   
1  Station2_LOG00002    SRV011           CL-7 2025-08-02 09:19:00   
2  Station2_LOG00003    SRV081           CL-4 2025-08-02 07:57:00   
3  Station2_LOG00004    SRV023           CL-1 2025-08-01 16:51:00   
4  Station2_LOG00005    SRV034           CL-6 2025-08-02 22:51:00   

   CPU_Utilization  Memory_Usage    Disk_IO  Network_Traffic_In_MB/s  \
0            40.91        42.640  54.580000                     5.72   
1            10.70        45.405  51.243333                     7.42   
2            59.41        48.170  47.906667                     3.99   
3            45.58        59.190  44.570000                    16.50   
4            55.32        20.390  16.510000                    20.19   

   Network_Traffic_Out_MB/s  Uptime_Hours  Downtime_Hours  
0                     10.61        454.16            8.71  
1                      7.13     

##3 Data Transformation

###3.1 Data Transformation for station1

####Categorizing the CPU_Utilization

In [ ]:
cpu =[
    station1["CPU_Utilization"]<40,
    (station1['CPU_Utilization']>=40) & (station1["CPU_Utilization"]<70),
    (station1['CPU_Utilization']>=70) & (station1["CPU_Utilization"]<90)

]
condition = ["Low","Medium","High"]
station1.loc[:,"CPU_Category"] = np.select(cpu,condition,default="Critical")


####Categorizing the Memory_Usage

In [ ]:
memory =[
    station1["Memory_Usage"]<50,
    station1["Memory_Usage"]<80
]
category = [
    "Good","Warning"
]
station1.loc[:,"Memory_Usage_Category"] = np.select(memory,category,default="Critical")

####Categorizing the Disk_IO

In [ ]:

disk =[
    station1['Disk_IO']<40,
    (station1['Disk_IO']>=40) & (station1['Disk_IO']<70),
    (station1['Disk_IO']>=70) & (station1['Disk_IO']<90)

]
condition = ["Low","Medium","High"]
station1.loc[:,"Disk_Category"] = np.select(disk,condition,default="Critical")



####Calculating Availability

In [ ]:
station1.loc[:,"Availability"] = np.where((station1["Uptime_Hours"] + station1["Downtime_Hours"])>0, (station1["Uptime_Hours"]/(station1["Uptime_Hours"]+ station1["Downtime_Hours"]))*100,0)

In [ ]:
print(station1.head())

              Log_ID Server_ID Server_Cluster       Log_Timestamp  \
0  Station1_LOG00001    SRV001           CL-8 2025-08-02 23:30:00   
1  Station1_LOG00002    SRV005           CL-3 2025-08-01 03:12:00   
2  Station1_LOG00003    SRV001           CL-8 2025-08-01 01:33:00   
3  Station1_LOG00004    SRV091           CL-2 2025-08-02 12:56:00   
4  Station1_LOG00005    SRV089           CL-1 2025-08-02 02:50:00   

   CPU_Utilization  Memory_Usage  Disk_IO  Network_Traffic_In_MB/s  \
0            94.63         81.30    44.10                     1.67   
1            34.98         45.56    58.36                    29.42   
2            37.41         73.65    74.51                     2.79   
3            86.86         60.65    23.75                    22.67   
4            91.59         62.23    50.36                    11.64   

   Network_Traffic_Out_MB/s  Uptime_Hours  Downtime_Hours CPU_Category  \
0                    21.790        506.74            2.77     Critical   
1               

####Calculating Resource efficiency

In [ ]:
station1["Resource_Efficiency (%)"] = (
    (station1["Availability"] * 0.40) +
    ((100 - abs(station1["CPU_Utilization"] - 65)) * 0.25) +
    ((100 - abs(station1["Memory_Usage"] - 65)) * 0.20) +
    ((100 - abs(station1["Disk_IO"] - 50)) * 0.15)
)


In [ ]:
print(station1.head())

              Log_ID Server_ID Server_Cluster       Log_Timestamp  \
0  Station1_LOG00001    SRV001           CL-8 2025-08-02 23:30:00   
1  Station1_LOG00002    SRV005           CL-3 2025-08-01 03:12:00   
2  Station1_LOG00003    SRV001           CL-8 2025-08-01 01:33:00   
3  Station1_LOG00004    SRV091           CL-2 2025-08-02 12:56:00   
4  Station1_LOG00005    SRV089           CL-1 2025-08-02 02:50:00   

   CPU_Utilization  Memory_Usage  Disk_IO  Network_Traffic_In_MB/s  \
0            94.63         81.30    44.10                     1.67   
1            34.98         45.56    58.36                    29.42   
2            37.41         73.65    74.51                     2.79   
3            86.86         60.65    23.75                    22.67   
4            91.59         62.23    50.36                    11.64   

   Network_Traffic_Out_MB/s  Uptime_Hours  Downtime_Hours CPU_Category  \
0                    21.790        506.74            2.77     Critical   
1               

In [ ]:
station1["Station"]='Station1'

###3.2 Data Transformation for station2

####Categorizing the CPU_Utilization

In [ ]:
cpu =[
    station2["CPU_Utilization"]<40,
    (station2['CPU_Utilization']>=40) & (station2["CPU_Utilization"]<70),
    (station2['CPU_Utilization']>=70) & (station2["CPU_Utilization"]<90)

]
condition = ["Low","Medium","High"]
station2.loc[:,"CPU_Category"] = np.select(cpu,condition,default="Critical")


####Categorizing the Memory_Usage

In [ ]:
memory =[
    station2["Memory_Usage"]<50,
    station2["Memory_Usage"]<80
]
category = [
    "Good","Warning"
]
station2.loc[:,"Memory_Usage_Category"] = np.select(memory,category,default="Critical")

####Categorizing the Disk_IO

In [ ]:

disk =[
    station2['Disk_IO']<40,
    (station2['Disk_IO']>=40) & (station2['Disk_IO']<70),
    (station2['Disk_IO']>=70) & (station2['Disk_IO']<90)

]
condition = ["Low","Medium","High"]
station2.loc[:,"Disk_Category"] = np.select(disk,condition,default="Critical")



####Calculating Availability

In [ ]:
station2.loc[:,"Availability"] = np.where((station2["Uptime_Hours"] + station2["Downtime_Hours"])>0, (station2["Uptime_Hours"]/(station2["Uptime_Hours"]+ station2["Downtime_Hours"]))*100,0)

In [ ]:
print(station1.head())

              Log_ID Server_ID Server_Cluster       Log_Timestamp  \
0  Station1_LOG00001    SRV001           CL-8 2025-08-02 23:30:00   
1  Station1_LOG00002    SRV005           CL-3 2025-08-01 03:12:00   
2  Station1_LOG00003    SRV001           CL-8 2025-08-01 01:33:00   
3  Station1_LOG00004    SRV091           CL-2 2025-08-02 12:56:00   
4  Station1_LOG00005    SRV089           CL-1 2025-08-02 02:50:00   

   CPU_Utilization  Memory_Usage  Disk_IO  Network_Traffic_In_MB/s  \
0            94.63         81.30    44.10                     1.67   
1            34.98         45.56    58.36                    29.42   
2            37.41         73.65    74.51                     2.79   
3            86.86         60.65    23.75                    22.67   
4            91.59         62.23    50.36                    11.64   

   Network_Traffic_Out_MB/s  Uptime_Hours  Downtime_Hours CPU_Category  \
0                    21.790        506.74            2.77     Critical   
1               

####Calculating Resource efficiency

In [ ]:
station2["Resource_Efficiency"] = (
    (station2["Availability"] * 0.40) +
    ((100 - abs(station2["CPU_Utilization"] - 65)) * 0.25) +
    ((100 - abs(station2["Memory_Usage"] - 65)) * 0.20) +
    ((100 - abs(station2["Disk_IO"] - 50)) * 0.15)
)


In [ ]:
print(station2.head())

              Log_ID Server_ID Server_Cluster       Log_Timestamp  \
0  Station2_LOG00001    SRV011           CL-7 2025-08-01 10:53:00   
1  Station2_LOG00002    SRV011           CL-7 2025-08-02 09:19:00   
2  Station2_LOG00003    SRV081           CL-4 2025-08-02 07:57:00   
3  Station2_LOG00004    SRV023           CL-1 2025-08-01 16:51:00   
4  Station2_LOG00005    SRV034           CL-6 2025-08-02 22:51:00   

   CPU_Utilization  Memory_Usage    Disk_IO  Network_Traffic_In_MB/s  \
0            40.91        42.640  54.580000                     5.72   
1            10.70        45.405  51.243333                     7.42   
2            59.41        48.170  47.906667                     3.99   
3            45.58        59.190  44.570000                    16.50   
4            55.32        20.390  16.510000                    20.19   

   Network_Traffic_Out_MB/s  Uptime_Hours  Downtime_Hours CPU_Category  \
0                     10.61        454.16            8.71       Medium   
1   

In [ ]:
station2["Station"]='Station2'

####Merging Cleaned Files(Fact_Table_Creation)

In [ ]:
combined = pd.concat([station1, station2], ignore_index=True)


In [ ]:
enrich = pd.merge(
    left=combined,
    right=metadata[['Server_ID', 'Server_Location', 'OS_Type', 'Admin_Name', 'Server_Cluster']],
    on='Server_ID',
    how='left'
)

##4 Data Loading

In [ ]:
combined.to_csv("Dimension_Table", index=False)


In [ ]:
metadata.to_csv("Fact_Table.csv", index=False)
